In [1]:

import joblib as jb
import pandas as pd
import numpy as np
import re
import os
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['svg.fonttype'] = 'none'
plt.rcParams['pdf.fonttype'] = 42
import basty.project.experiment_processing as experiment_processing
import seaborn as sns
sns.set_theme(style = 'dark')
import datetime
from dateutil import parser
import matplotlib.dates as mdates

from pathlib import Path
from basty.utils import misc

import sys
sys.path.append("/home/grover/Desktop/gitDLC/basty")

In [2]:
CONFIG_PATH = r"Z:\mfk\basty-projects\main_cfg.yaml"
project = experiment_processing.Project(
   CONFIG_PATH
)
FPS = project.fps
FIG_PATH = r'C:\Users\Grover\Documents\GitHub\deepsleepfigs'

In [3]:
data_path_dict = project.data_path_dict
df = misc.parse_experiment_names(data_path_dict)
flysexpath = r'Y:\DeepSleepPaperData\flysexinfo.yaml'
df = misc.update_expt_info_df(df,flysexpath)

In [4]:
df

,ExptNames,Path,SD,Age,Sex,Experimenter
0,Fly01182022_F_A_8am,Y:\DeepSleepPaperData\Annotated\WT\Fly01182022...,False,5.0,F,MK
1,Fly01192022_F_B_8am,Y:\DeepSleepPaperData\Annotated\WT\Fly01192022...,False,5.0,F,MK
2,Fly01202022_F_A_8am,Y:\DeepSleepPaperData\Annotated\WT\Fly01202022...,False,5.0,F,MK
3,Fly01232022_M_B_8am,Y:\DeepSleepPaperData\Annotated\WT\Fly01232022...,False,5.0,M,MK
4,Fly01242022_F_A_8am,Y:\DeepSleepPaperData\Annotated\WT\Fly01242022...,False,5.0,F,MK
...,...,...,...,...,...,...
72,Fly08182021_F_B_6d_8am,Y:\DeepSleepPaperData\Annotated\WT\Fly08182021...,False,6.0,F,MK
73,Fly11052021_F_A_SD_5D_8am,Y:\DeepSleepPaperData\Annotated\SD\Fly11052021...,True,5.0,F,MK
74,Fly11152020_F_SD,Y:\DeepSleepPaperData\Annotated\SD\Fly11152020...,True,5.0,F,MK
75,Fly11192021_F_A_SD_5D_9am,Y:\DeepSleepPaperData\Annotated\SD\Fly11192021...,True,5.0,F,MK


In [5]:
llh = misc.get_likelihood(data_path_dict)

In [7]:
results_folder = r"Z:\mfk\basty-projects\backup-allnohaltere\results\semisupervised_pair_kNN\predictions.15NN.neighbor_weights-distance.neighbor_weights_norm-log_count.activation-standard.voting-soft.voting_weights-None\exports"

In [8]:
csv_files = [file for file in os.listdir(results_folder) if file.endswith('.csv')]

In [9]:
dfs = list()
for file in csv_files:
    data = pd.read_csv(os.path.join(results_folder,file))
    data['ExptNames'] = os.path.splitext(file)[0]
    dfs.append(data)

df_data = pd.concat(dfs, ignore_index = True)
df_data = df_data.rename(columns = {'Unnamed: 0': 'Idx'})

In [10]:
#Remove unnecessary columns
df_data.drop(['HaltereSwitch','Noise'],axis=1,inplace=True)

In [11]:
llh  = llh[llh['ExptNames'].isin(df_data.ExptNames.unique())]

In [12]:
BEHAVIORS = [
'Idle&Other',
'PosturalAdjustment&Moving',
'Feeding',
'Grooming',
'ProboscisPumping']

In [13]:
newDf = pd.merge(df_data,df,left_on = 'ExptNames',right_on = 'ExptNames')

In [14]:
newDf

,Idx,Idle&Other,PosturalAdjustment&Moving,Feeding,Grooming,ProboscisPumping,ExptNames,Path,SD,Age,Sex,Experimenter
0,0,1.0,0.0,0.0,0.0,0.0,Fly05182022_5d,Y:\DeepSleepPaperData\Data\WT\Fly05182022_5d,False,5.0,M,MK
1,1,1.0,0.0,0.0,0.0,0.0,Fly05182022_5d,Y:\DeepSleepPaperData\Data\WT\Fly05182022_5d,False,5.0,M,MK
2,2,1.0,0.0,0.0,0.0,0.0,Fly05182022_5d,Y:\DeepSleepPaperData\Data\WT\Fly05182022_5d,False,5.0,M,MK
3,3,1.0,0.0,0.0,0.0,0.0,Fly05182022_5d,Y:\DeepSleepPaperData\Data\WT\Fly05182022_5d,False,5.0,M,MK
4,4,1.0,0.0,0.0,0.0,0.0,Fly05182022_5d,Y:\DeepSleepPaperData\Data\WT\Fly05182022_5d,False,5.0,M,MK
...,...,...,...,...,...,...,...,...,...,...,...,...
69652104,647995,1.0,0.0,0.0,0.0,0.0,Fly08032022_6d_SD_B,Y:\DeepSleepPaperData\Data\SD\Fly08032022_6d_SD_B,True,6.0,F,CB
69652105,647996,1.0,0.0,0.0,0.0,0.0,Fly08032022_6d_SD_B,Y:\DeepSleepPaperData\Data\SD\Fly08032022_6d_SD_B,True,6.0,F,CB
69652106,647997,1.0,0.0,0.0,0.0,0.0,Fly08032022_6d_SD_B,Y:\DeepSleepPaperData\Data\SD\Fly08032022_6d_SD_B,True,6.0,F,CB
69652107,647998,1.0,0.0,0.0,0.0,0.0,Fly08032022_6d_SD_B,Y:\DeepSleepPaperData\Data\SD\Fly08032022_6d_SD_B,True,6.0,F,CB


In [15]:
wt_all = newDf[newDf['SD']==False]
male_wt = wt_all[wt_all['Sex'] == 'M']
female_wt = wt_all[wt_all['Sex'] == 'F']